# Read in e3t and create a +/-2 m SSH versions 

In [1]:
import xarray as xr
import numpy as np
import time
from datetime import datetime, timedelta
from dateutil.parser import parse
import os
from netCDF4 import Dataset

## User input

In [2]:
date_begin = parse('5 june 2015')
date_end   = parse('12 june 2015')
path       = '/results2/SalishSea/nowcast-green.201806/'
filetype   = 'carp_T'
depth_change = 2
out_e3t_frac = '/home/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/vvl/e3t_frac_dz_2.nc'

# pick Salmon Bank location [256,265], but remember that MOHID is transposed! such that SSC [yloc_ssc,xloc_ssc]-> [xloc_ssc,yloc_ssc]_mohid = [yloc_mohid,xloc_mohid]
yloc_mohid       = 265
xloc_mohid       = 256

In [3]:
def mung_array(SSC_gridded_array, array_slice_type):
    """Transform an array containing SalishSeaCast-gridded data and transform it 
       into a MOHID-gridded array by:
         1) Cutting off the grid edges
         2) Transposing the X and Y axes
         3) Flipping the depth dimension, if it is present
         4) Converting the NaNs to 0
    
        :arg SSC_gridded_array: SalishSeaCast-gridded array
        :type numpy.ndarray: :py:class:'ndarray'
    
        :arg array_slice_type: str, one of '2D' or '3D'
        :type str: :py:class:'str'

        :return MOHID_gridded_array: MOHID-gridded array produced by applying operation
                                     1-4 on SSC_gridded_array
        :type numpy.ndarray: :py:class:'ndarray'
    """
    shape = SSC_gridded_array.shape
    ndims = len(shape)
    assert(array_slice_type in  ('2D', '3D')), f"Invalid option {array_slice_type}. array_slice_type must be one of ('2D', '3D')"
    if array_slice_type is '2D':
        assert(ndims in (2,3)), f'The shape of the array given is {shape}, while the option chosen was {array_slice_type}'
        if ndims == 2:
            MOHID_gridded_array = SSC_gridded_array[1:897:,1:397]
            del(SSC_gridded_array)
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [1,0])
        else:
            MOHID_gridded_array = SSC_gridded_array[...,1:897:,1:397]
            del(SSC_gridded_array)
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,2,1])

    else:
        assert(ndims in (3,4)), f'The shape of the array given is {shape}, while the option chosen was {array_slice_type}'
        MOHID_gridded_array = SSC_gridded_array[...,1:897:,1:397]
        del(SSC_gridded_array)
        if ndims == 3:
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,2,1])
            MOHID_gridded_array = np.flip(MOHID_gridded_array, axis = 0)
        else:
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,1,3,2])
            MOHID_gridded_array = np.flip(MOHID_gridded_array, axis = 1)

    MOHID_gridded_array = np.nan_to_num(MOHID_gridded_array).astype('float64')

    return MOHID_gridded_array

## Ashu's function for writing HDF5 file

In [4]:
def write_grid(data, datearrays, metadata, filename, groupname, accumulator, compression_level):
    shape = data[0].shape
    with h5py.File(filename) as f:
        time_group = f.get('/Time')
        if time_group is None:
            time_group = f.create_group('/Time')
        data_group_path = f'/Results/{groupname}'
        data_group = f.get(data_group_path)
        if data_group is None:
            data_group = f.create_group(data_group_path)

        for i, datearray in enumerate(datearrays):
            numeric_attribute = ((5 - len(str(i + accumulator))) * '0') + str(i + accumulator)
            child_name = 'Time_' + numeric_attribute
            timestamp = time_group.get(child_name)
            if timestamp is None:
                dataset = time_group.create_dataset(
                    child_name,
                    shape = (6,),
                    data = datearray,
                    chunks = (6,),
                    compression = 'gzip',
                    compression_opts = compression_level
                    )
                time_metadata = {
                    'Maximum' : np.array(datearray[0]),
                    'Minimum' : np.array([-0.]),
                    'Units' : b'YYYY/MM/DD HH:MM:SS'
                    }
                dataset.attrs.update(time_metadata)
            else:
                assert (np.asarray(timestamp) == datearray).all(), f'Time record {child_name} exists and does not match with {datearray}'

            child_name = groupname + '_' + numeric_attribute
            if data_group.get(child_name) is not None:
                print(f'Dataset already exists at {child_name}')
            else:
                dataset = data_group.create_dataset(
                    child_name,
                    shape = shape,
                    data = data[i],
                    chunks = shape,
                    compression = 'gzip',
                    compression_opts = compression_level
                    )
                dataset.attrs.update(metadata)

## Generate list of dates from user input

In [5]:
daterange = [date_begin, date_end]
# append all filename strings within daterange to lists
e3t_list = []
for day in range(np.diff(daterange)[0].days + 1):
    datestamp = daterange[0] + timedelta(days = day)
    datestr1 = datestamp.strftime('%d%b%y').lower()
    datestr2 = datestamp.strftime('%Y%m%d')

    # check if file exists. exit if it does not. add path to list if it does.
    file_path = f'{path}{datestr1}/SalishSea_1h_{datestr2}_{datestr2}_{filetype}.nc'
    if not os.path.exists(file_path):
        print(f'File {file_path} not found. Check Directory and/or Date Range.')
    e3t_list.append(file_path)

In [6]:
e3t_list

['/results2/SalishSea/nowcast-green.201806/05jun15/SalishSea_1h_20150605_20150605_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/06jun15/SalishSea_1h_20150606_20150606_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/07jun15/SalishSea_1h_20150607_20150607_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/08jun15/SalishSea_1h_20150608_20150608_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/09jun15/SalishSea_1h_20150609_20150609_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/10jun15/SalishSea_1h_20150610_20150610_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/11jun15/SalishSea_1h_20150611_20150611_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/12jun15/SalishSea_1h_20150612_20150612_carp_T.nc']

### Create mask

In [7]:
mask = mung_array(xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DMeshMaskV17-02').isel(time = 0).tmask.values, '3D')

### Test process with one file

In [8]:
data = xr.open_dataset(e3t_list[0])
datetimelist = data.time_counter.values.astype('datetime64[s]').astype(datetime)
datearrays = [np.array(
        [d.year, d.month, d.day, d.hour, d.minute,d.second]
        ).astype('float64') for d in datetimelist]
del(datetimelist)
e3t = data.e3t.values
e3t = mung_array(e3t, '3D')
e3t = e3t*mask
metadata = {
   'FillValue' : np.array([0.]),
   'Units' : b'?C'
   }

#### Create a matrix of %depth values for all locations and times by looping through time and space

In [10]:
total_depth = e3t.sum(1)
e3t_frac_dz = np.empty_like(e3t)

if os.path.isfile('/home/rmueller/data/vvl/test_e3t_frac.nc'): 
    test = xr.open_dataset('/home/rmueller/data/vvl/test_e3t_frac.nc')
    print('Loading e3t_frac_dz from file')
else:
    print('Creating matrix of percent total depth for e3t levels (this will take some time)')
    for t in range(e3t.shape[0]):
        for i in range(e3t.shape[2]):
            for j in range(e3t.shape[3]):
                for z in range(e3t.shape[1]):
                    e3t_frac_dz[t,z,i,j] = e3t[t,z,i,j]/total_depth[t,i,j]
                    
    print('saving to ', out_e3t_frac)
    # convert to xarray for ease of output
    xrfrac = xr.DataArray(e3t_frac_dz)
    xrfrac.to_netcdf('/home/rmueller/data/vvl/test_e3t_frac.nc')

Loading e3t_frac_dz from file


In [11]:
# Calculate new e3t based on desired depth change
e3t_new = (total_depth[1,256,265] + depth_change) * e3t_frac_dz

In [13]:
e3t_new.shape

(24, 40, 396, 896)

## Test process with multiple files

In [14]:
for file_path in e3t_list:
    data = xr.open_dataset(file_path)
    datetimelist = data.time_counter.values.astype('datetime64[s]').astype(datetime)
    datearrays = [np.array(
            [d.year, d.month, d.day, d.hour, d.minute,d.second]
            ).astype('float64') for d in datetimelist]
    del(datetimelist)
    e3t = data.e3t.values
    e3t = mung_array(e3t, '3D')
    e3t = e3t*mask
    metadata = {
       'FillValue' : np.array([0.]),
       'Units' : b'?C'
       }

In [15]:
e3t.shape

(24, 40, 396, 896)